## Imports

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from scipy.optimize import minimize
from scipy.stats import norm

## Data Setup

In [2]:
import pandas as pd

df_happiness = (
    pd.read_csv('https://tinyurl.com/worldhappiness2018')
    .dropna()
    .rename(columns = {'happiness_score': 'happiness'})
    .filter(regex = '_sc|country|happ')
)

## Prediction Error

MSE comparison between the two models

In [3]:
y = df_happiness['happiness']

# Calculate the error for the guess of four
prediction = np.min(df_happiness['happiness']) + 1 * df_happiness['life_exp_sc']
mse_model_A   = np.mean((y - prediction)**2)

# Calculate the error for our other guess
prediction = y.mean() + .5 * df_happiness['life_exp_sc']
mse_model_B  = np.mean((y - prediction)**2)

pd.DataFrame({
    'Model': ['A', 'B'],
    'MSE': [mse_model_A, mse_model_B]
})

,Model,MSE
0,A,5.086298
1,B,0.637560


## OLS

In [4]:
# for later comparison
model_lr_happy = smf.ols('happiness ~ life_exp_sc', data = df_happiness).fit()

def ols(par, X, y):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]

    # Calculate the predicted values
    y_hat = X @ par  # @ is matrix multiplication
    
    # Calculate the mean of the squared errors
    value = np.mean((y - y_hat)**2)
    
    # Return the objective value
    return value

In [5]:
from itertools import product

guesses = pd.DataFrame(
    product(
        np.arange(1, 7, 0.1),
        np.arange(-1, 1, 0.1)
    ),
    columns = ['b0', 'b1']
)

# Example for one guess
ols(
    par = guesses.iloc[0,:],
    X = df_happiness['life_exp_sc'],
    y = df_happiness['happiness']
)

23.77700449624871

In [6]:
guesses['objective'] = guesses.apply(
    lambda x: ols(
        par = x, 
        X = df_happiness['life_exp_sc'], 
        y = df_happiness['happiness']
    ),
    axis = 1
)

min_loss = guesses[guesses['objective'] == guesses['objective'].min()]

min_loss

,b0,b1,objective
899,5.4,0.9,0.490675


In [7]:
model_lr_happy_life = sm.OLS(df_happiness['happiness'], sm.add_constant(df_happiness['life_exp_sc'])).fit()

model_lr_happy_life.params, model_lr_happy_life.scale

(const          5.444832
 life_exp_sc    0.887796
 dtype: float64,
 0.4973994106686574)

## Optimization

In [8]:
our_ols_optim = minimize(
    fun  = ols,
    x0   = np.array([1., 0.]),
    args = (
        np.array(df_happiness['life_exp_sc']), 
        np.array(df_happiness['happiness'])
    ),
    method  = 'BFGS',   # optimization algorithm
    tol     = 1e-6,     # tolerance
    options = {
        'maxiter': 500  # max iterations
    }
)

our_ols_optim

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.48851727833528863
        x: [ 5.445e+00  8.878e-01]
      nit: 5
      jac: [-7.451e-09  0.000e+00]
 hess_inv: [[ 5.000e-01  3.539e-06]
            [ 3.539e-06  5.045e-01]]
     nfev: 21
     njev: 7

## Maximum Likelihood

Initial exploration

In [9]:
# two example life expectancy scores, at the mean (0) and 1 sd above
life_expectancy = np.array([0, 1])

# observed happiness scores
happiness = np.array([4, 5.2])

# predicted happiness with rounded coefs
mu = 5 + 1 * life_expectancy

# just a guess for sigma
sigma = .5

# likelihood for each observation
L = norm.pdf(happiness, loc = mu, scale = sigma)
L

array([0.10798193, 0.22184167])

Main function

In [10]:
def max_likelihood(par, X, y):
    
    # setup
    X = np.c_[np.ones(X.shape[0]), X] # add a column of 1s for the intercept
    beta   = par[1:]         # coefficients
    sigma  = np.exp(par[0])  # error sd, exp keeps positive
    N = X.shape[0]

    LP = X @ beta            # linear predictor
    mu = LP                  # identity link in the glm sense

    # calculate (log) likelihood
    ll = norm.logpdf(y, loc = mu, scale = sigma)

    value = -np.sum(ll)      # negative for minimization

    return value

our_max_like = minimize(
    fun  = max_likelihood,
    x0   = np.array([1, 0, 0]),
    args = (
        np.array(df_happiness['life_exp_sc']), 
        np.array(df_happiness['happiness'])
    )
)

our_max_like['x']

array([-0.35819022,  5.44483214,  0.88779604])

## Penalized Objectives

In [11]:
# we use lambda_ because lambda is a reserved word in python
def ridge(par, X, y, lambda_ = 0):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]

    # Calculate the predicted values
    mu = X @ par
    
    # Calculate the error
    value = np.sum((y - mu)**2)
    
    # Add the penalty
    value = value + lambda_ * np.sum(par**2)
    
    return value

our_ridge = minimize(
    fun  = ridge,
    x0   = np.array([0, 0, 0, 0]),
    args = (
        np.array(df_happiness.drop(columns=['happiness', 'country'])),
        np.array(df_happiness['happiness']), 
        0.1
    )
)

In [12]:
our_ridge['x']

array([ 5.439975  ,  0.52422716, -0.1053189 ,  0.43749604])

## Classification

### Misclassification Error

In [13]:
def misclassification_rate(par, X, y, class_threshold = .5):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]

    # Calculate the 'linear predictor'
    mu = X @ par 
    
    # Convert to a probability ('sigmoid' function)
    p = 1 / (1 + np.exp(-mu))
    
    # Convert to a class
    predicted_class = np.where(p > class_threshold, 1, 0)
    
    # Calculate the mean error
    value = np.mean(y - predicted_class)

    return value

### Log Loss

In [14]:
def log_loss(par, X, y):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]

    # Calculate the predicted values
    y_hat = X @ par
    
    # Convert to a probability ('sigmoid' function)
    y_hat = 1 / (1 + np.exp(-y_hat))
    
    # likelihood
    ll = y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)

    value = -np.sum(ll)
    
    return value

In [15]:
df_happiness_bin = df_happiness.copy()
df_happiness_bin['happiness'] = np.where(df_happiness['happiness'] > 5.5, 1, 0)

model_logloss = minimize(
    log_loss,
    x0 = np.array([0, 0, 0, 0]),
    args = (
        df_happiness_bin[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']],
        df_happiness_bin['happiness']
    )
)

model_glm = smf.glm(
    'happiness ~ life_exp_sc + corrupt_sc + gdp_pc_sc',
    data   = df_happiness_bin,
    family = sm.families.Binomial()
).fit()

model_logloss['x']

array([-0.16365245,  1.81715104, -0.46478325,  1.13108169])

## Gradient Descent

In [16]:
def gradient_descent(
    par, 
    X, 
    y, 
    tolerance = 1e-3, 
    maxit = 1000, 
    learning_rate = 1e-3
):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]
    
    # initialize
    beta = par
    loss = np.sum((X @ beta - y)**2)
    tol = 1
    iter = 1

    while (tol > tolerance and iter < maxit):
        LP = X @ beta
        grad = X.T @ (LP - y)
        betaCurrent = beta - learning_rate * grad
        tol = np.max(np.abs(betaCurrent - beta))
        beta = betaCurrent
        loss = np.append(loss, np.sum((LP - y)**2))
        iter = iter + 1

    output = {
        'par': beta,
        'loss': loss,
        'MSE': np.mean((LP - y)**2),
        'iter': iter,
        'fitted': LP
    }

    return output

our_gd = gradient_descent(
    par = np.array([0, 0, 0, 0]),
    X = df_happiness[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']].to_numpy(),
    y = df_happiness['happiness'].to_numpy(),
    learning_rate = 1e-3
)

In [17]:
our_gd['par']

array([ 5.43691264,  0.52121949, -0.10746734,  0.43896778])

## SGD

In [18]:
def stochastic_gradient_descent(
    par, # parameter estimates
    X,   # model matrix
    y,   # target variable
    learning_rate = 1, # the learning rate
    stepsize_tau = 0,  # if > 0, a check on the LR at early iterations
    average = False    # a variation of the approach
):
    # initialize
    np.random.seed(1234)

    # shuffle the data
    idx = np.random.choice(
        df_happiness.shape[0], 
        df_happiness.shape[0], 
        replace = False
    )
    X = X[idx, :]
    y = y[idx]
    
    X = np.c_[np.ones(X.shape[0]), X]
    beta = par

    # Collect all estimates
    betamat = np.zeros((X.shape[0], beta.shape[0]))

    # Collect fitted values at each point))
    fits = np.zeros(X.shape[0])

    # Collect loss at each point
    loss = np.zeros(X.shape[0])

    # adagrad per parameter learning rate adjustment
    s = 0

    # a smoothing term to avoid division by zero
    eps = 1e-8

    for i in range(X.shape[0]):
        Xi = X[None, i, :]
        yi = y[i]

        # matrix operations not necessary here,
        # but makes consistent with previous gd func
        LP = Xi @ beta
        grad = Xi.T @ (LP - yi)
        s = s + grad**2 # adagrad approach

        # update
        beta = beta - learning_rate / \
            (stepsize_tau + np.sqrt(s + eps)) * grad

        betamat[i, :] = beta

        fits[i] = LP
        loss[i] = np.sum((LP - yi)**2)

    LP = X @ beta
    lastloss = np.sum((LP - y)**2)

    output = {
        'par': beta,          # final estimates
        'par_chain': betamat, # estimates at each iteration
        'MSE': lastloss / X.shape[0],
        'predictions': LP
    }

    return output

X = df_happiness[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']].to_numpy()
y = df_happiness['happiness'].to_numpy()

our_sgd = stochastic_gradient_descent(
    par = np.array([np.mean(df_happiness['happiness']), 0, 0, 0]),
    X = X,
    y = y,
    learning_rate = .15,
    stepsize_tau = .1
)

our_sgd['par'], our_sgd['MSE']

/var/folders/x6/4jhswqxj0sqf_gkgq6lw6l880000gn/T/ipykernel_38163/2270994482.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fits[i] = LP


(array([ 5.42765734,  0.53391505, -0.15014284,  0.39964098]),
 0.36857925293717886)

## Uncertainty

### Frequentist

In [19]:
model = smf.ols(
    'happiness ~ life_exp_sc + corrupt_sc + gdp_pc_sc',
    data = df_happiness
).fit()

model.conf_int()

model.get_prediction().summary_frame() # both 'confidence' and 'prediction' intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,3.987671,0.132758,3.724521,4.250820,2.736966,5.238375
1,5.496638,0.104065,5.290363,5.702914,4.256653,6.736624
2,5.676520,0.087470,5.503139,5.849901,4.441580,6.911459
3,5.406585,0.107000,5.194492,5.618678,4.165618,6.647552
4,6.966640,0.126756,6.715389,7.217892,5.718385,8.214896
...,...,...,...,...,...,...
107,5.861256,0.077897,5.706850,6.015661,4.628837,7.093674
108,5.290368,0.147161,4.998669,5.582067,4.033347,6.547389
109,5.327998,0.083659,5.162170,5.493825,4.094096,6.561899
110,4.308105,0.101039,4.107828,4.508383,3.069103,5.547107


In [20]:
model.get_prediction().summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,3.987671,0.132758,3.724521,4.250820,2.736966,5.238375
1,5.496638,0.104065,5.290363,5.702914,4.256653,6.736624
2,5.676520,0.087470,5.503139,5.849901,4.441580,6.911459
3,5.406585,0.107000,5.194492,5.618678,4.165618,6.647552
4,6.966640,0.126756,6.715389,7.217892,5.718385,8.214896
...,...,...,...,...,...,...
107,5.861256,0.077897,5.706850,6.015661,4.628837,7.093674
108,5.290368,0.147161,4.998669,5.582067,4.033347,6.547389
109,5.327998,0.083659,5.162170,5.493825,4.094096,6.561899
110,4.308105,0.101039,4.107828,4.508383,3.069103,5.547107


### Monte Carlo

In [21]:
# we'll use the model from the previous section
model = smf.ols(
    'happiness ~ life_exp_sc + corrupt_sc + gdp_pc_sc',
    data = df_happiness
).fit()

def mc_predictions(model, nsim=2500, seed=42):
    np.random.seed(seed)

    params_est = model.params
    params = np.random.multivariate_normal(
        mean = params_est,
        cov = model.cov_params(),
        size = nsim
    )

    sigma = model.mse_resid**.5
    X = model.model.exog

    y_hat = X @ params.T + np.random.normal(scale = sigma, size = (X.shape[0], nsim))

    pred_int = np.quantile(y_hat, q = [.025, .975], axis = 1)

    return pred_int

our_mc = mc_predictions(model)

In [22]:
# Statsmodels Prediction Intervals
prediction_intervals = model.get_prediction().summary_frame()
statsmodels_lower = prediction_intervals['obs_ci_lower']
statsmodels_upper = prediction_intervals['obs_ci_upper']


pd.DataFrame({
    'observed_value': df_happiness['happiness'],
    'prediction': model.fittedvalues,
    'simulated_lower': our_mc[0],
    'simulated_upper': our_mc[1],
    'statsmodels_lower': statsmodels_lower,
    'statsmodels_upper': statsmodels_upper
}).round(3)

,observed_value,prediction,simulated_lower,simulated_upper,statsmodels_lower,statsmodels_upper
0,3.632,3.988,2.770,5.197,2.737,5.238
1,4.586,5.497,4.278,6.759,4.257,6.737
2,6.388,5.677,4.451,6.889,4.442,6.911
3,4.321,5.407,4.218,6.666,4.166,6.648
4,7.272,6.967,5.733,8.167,5.718,8.215
...,...,...,...,...,...,...
107,6.379,5.861,4.670,7.090,4.629,7.094
108,6.096,5.290,4.057,6.547,4.033,6.547
109,4.806,5.328,4.126,6.530,4.094,6.562
110,4.377,4.308,3.097,5.531,3.069,5.547


### Bootstrap

In [23]:
def bootstrap(X, y, nboot=100, seed=123):
    # add a column of 1s for the intercept
    X = np.c_[np.ones(X.shape[0]), X]
    N = X.shape[0]

    # initialize
    beta = np.empty((nboot, X.shape[1]))
    
    # beta = pd.DataFrame(beta, columns=['Intercept'] + list(cn))
    mse = np.empty(nboot)    

    # set seed
    np.random.seed(seed)

    for i in range(nboot):
        # sample with replacement
        idx = np.random.randint(0, N, N)
        Xi = X[idx, :]
        yi = y[idx]

        # estimate model
        model = LinearRegression(fit_intercept=False)
        mod = model.fit(Xi, yi)

        # save results
        beta[i, :] = mod.coef_
        mse[i] = np.sum((mod.predict(Xi) - yi)**2) / N

    # given mean estimates, calculate MSE
    y_hat = X @ beta.mean(axis=0)
    final_mse = np.sum((y - y_hat)**2) / N

    output = {
        'par': beta,
        'mse': mse,
        'final_mse': final_mse
    }

    return output

our_boot = bootstrap(
    X = df_happiness[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']],
    y = df_happiness['happiness'],
    nboot = 1000
)

In [24]:
np.percentile(our_boot['par'], 2.5, axis=0)

array([ 5.34092479,  0.27665819, -0.29543964,  0.19114177])

In [25]:
pd.DataFrame({
    'param': ['Intercept', 'life_exp_sc', 'corrupt_sc', 'gdp_pc_sc'],
    'mean': our_boot['par'].mean(axis=0),
    'lwr': np.percentile(our_boot['par'], 2.5, axis=0),
    'upr': np.percentile(our_boot['par'], 97.5, axis=0)
})

,param,mean,lwr,upr
0,Intercept,5.451703,5.340925,5.572782
1,life_exp_sc,0.511917,0.276658,0.754842
2,corrupt_sc,-0.106482,-0.295440,0.080125
3,gdp_pc_sc,0.459829,0.191142,0.776553


### Bayesian

In [26]:
from scipy.stats import beta

pk = np.array([
    'goal','goal','goal','miss','miss',
    'goal','goal','miss','goal','goal'
])

# convert to numeric, arbitrarily picking goal=1, miss=0
N = len(pk)                     # sample size
n_goal = np.sum(pk == 'goal')   # number of pk made
n_miss = np.sum(pk == 'miss')   # number of those miss

# grid of potential theta values
theta = np.linspace(1 / (N + 1), N / (N + 1), 10)

### prior distribution
# beta prior with mean = .5, but fairly diffuse
# examine the prior
# theta = beta.rvs(5, 5, size = 1000)
# plt.hist(theta, bins = 20, color = 'lightblue')
p_theta = beta.pdf(theta, 5, 5)

# Normalize so that values sum to 1
p_theta = p_theta / np.sum(p_theta)

# likelihood (binomial)
p_data_given_theta = np.math.comb(N, n_goal) * theta**n_goal * (1 - theta)**n_miss

# posterior (combination of prior and likelihood)
# marginal probability of the data used for normalization
p_data = np.sum(p_data_given_theta * p_theta) 

p_theta_given_data = p_data_given_theta * p_theta / p_data  # Bayes theorem

# final estimate
theta_est = np.sum(theta * p_theta_given_data)
theta_est

/var/folders/x6/4jhswqxj0sqf_gkgq6lw6l880000gn/T/ipykernel_38163/1473881323.py:27: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  p_data_given_theta = np.math.comb(N, n_goal) * theta**n_goal * (1 - theta)**n_miss


0.599999996503221

### Conformal

In [27]:
def split_conformal(X, y, new_data, alpha = .05, calibration_split = .5):
    # Splitting the data into training and calibration sets
    X_train, X_cal, y_train, y_cal = train_test_split(
        X, 
        y,
        test_size = calibration_split,
        random_state = 123
    )

    N = X_train.shape[0]

    # Train the base model
    model = LinearRegression().fit(X_train, y_train)

    # Calculate residuals on calibration set
    cal_preds = model.predict(X_cal)
    residuals = np.abs(y_cal - cal_preds)

    # Sort residuals and find the quantile corresponding to (1-alpha)
    residuals = np.sort(residuals)

    # The correction here is useful for small sample sizes
    quantile  = np.quantile(residuals, (1 - alpha) * (N / (N + 1)))

    # Make predictions on new data and calculate prediction intervals
    preds = model.predict(new_data)
    lower_bounds = preds - quantile
    upper_bounds = preds + quantile

    # Return predictions and prediction intervals
    return {
        'cp_error': quantile,
        'preds': preds,
        'lower_bounds': lower_bounds,
        'upper_bounds': upper_bounds
    }


In [28]:
# split data
from sklearn.model_selection import train_test_split

X = df_happiness[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']]
y = df_happiness['happiness']

X_train, X_test, y_train, y_test = train_test_split(
    df_happiness[['life_exp_sc', 'corrupt_sc', 'gdp_pc_sc']],
    df_happiness['happiness'],
    test_size = 0.5,
    random_state = 1234
)

our_cp_error = split_conformal(
    X_train,
    y_train,
    X_test,
    alpha = .1
)

print(our_cp_error['cp_error'])

pd.DataFrame({
    'preds': our_cp_error['preds'],
    'lower_bounds': our_cp_error['lower_bounds'],
    'upper_bounds': our_cp_error['upper_bounds']
}).head()

1.1358174413022732


,preds,lower_bounds,upper_bounds
0,4.669552,3.533734,5.805369
1,4.680675,3.544858,5.816493
2,6.325134,5.189317,7.460952
3,3.409876,2.274058,4.545693
4,4.448433,3.312615,5.584250


In [29]:
from mapie.regression import MapieRegressor

model = MapieRegressor(LinearRegression(), method = 'base', random_state=123)
y_pred, y_pis = model.fit(X_train, y_train).predict(X_test, alpha = 0.1)

# take the first difference between upper and lower bounds,
# since it's constant for all predictions in this setting

cp_error = (y_pis[0, 1, 0] - y_pis[0, 0, 0]) / 2  

In [30]:
y_pis[:5].reshape(-1, 2)

array([[3.93113326, 6.15153942],
       [3.93302836, 6.15343453],
       [5.08887412, 7.30928028],
       [2.7385576 , 4.95896377],
       [3.73161544, 5.95202161]])

In [31]:
our_cp_error['cp_error'], cp_error

(1.1358174413022732, 1.1102030815534594)